#WELCOME TO THE BACKTESTER!

You can test your trading strategy using historical data.
You get to choose the starting conditions, the assets you trade on etc.

First things first, run the "MAIN CODE" here below and scroll to a cell titled "CHOICE".

In [ ]:
#### "MAIN CODE" ####

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta as rd
import random
import copy
import yfinance as yf
from fredapi import Fred

class Trade:
    def __init__(self, asset, trade_type, price, quantity, date):
        '''
        asset: string e.g. "AMZN"
        trade_type: string, "buy" or "sell"
        price: int or float
        quantity: int or float
        date: pandas datetime object
        '''
        self.asset = asset
        self.trade_type = trade_type
        self.price = price
        self.quantity = quantity
        self.date = date

    def to_dict(self):
        return {
            "asset": self.asset,
            "type": self.trade_type,
            "price": self.price,
            "quantity": self.quantity,
            "date": self.date
        }

class Portfolio:
    def __init__(self, initial_cash, initial_positions):
        # Store initial positions
        self.initial_positions = copy.deepcopy(initial_positions)
        self.initial_value = initial_cash
        for asset in self.initial_positions:
            self.initial_value += self.initial_positions[asset]["quantity"] * self.initial_positions[asset]["price"]

        # Start portfolio variables
        self.cash = initial_cash
        self.positions = copy.deepcopy(initial_positions)
        
    def buy(self, asset, price, quantity):
        self.cash -= quantity * price
        if asset in self.positions:
           self.positions[asset]["quantity"] += quantity
           self.positions[asset]["price"] = price
        else:
            self.positions[asset] = {
                "quantity": quantity,
                "price": price
            }
        if self.positions[asset]["quantity"] == 0:
            del self.positions[asset]
    
    def sell(self, asset, price, quantity):
        self.cash += quantity * price
        if asset in self.positions:
           self.positions[asset]["quantity"] -= quantity
           self.positions[asset]["price"] = price
        else:
            self.positions[asset] = {
                "quantity": -quantity,
                "price": price
            }
        if self.positions[asset]["quantity"] == 0:
            del self.positions[asset]

class Strategy:
    def generate_signals(self, data):
        raise NotImplementedError("Override this in a subclass")

    def sma(self, data, window):
        """Calculate Simple Moving Average."""
        return data.rolling(window).mean()

    def ema(self, data, window):
        """Calculate Exponential Moving Average."""
        return data.ewm(span=window, adjust=False).mean()

class MovingAverageCrossover(Strategy):
    def __init__(self, short_window=20, long_window=50):
        super().__init__()
        self.short_window = short_window
        self.long_window = long_window

    def generate_signals(self, data):
        data["SMA_short"] = self.sma(data["Close"], self.short_window)
        data["SMA_long"] = self.sma(data["Close"], self.long_window)
        return (data["SMA_short"] > data["SMA_long"]).astype(int)

def random_date(start_date, end_date, max_dist):
    """Generate a random date between start_date and end_date, which are strings (inclusive). max_dist is the maximum possible distance that the
    generated date can have from the start date"""
    # Convert strings to datetime objects
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    if max_dist == None:
        delta_days = (end - start).days
    else:
        delta_days = min((end - start).days, max_dist, 0)
    random_days = random.randint(0, delta_days)
    return start + dt.timedelta(days=random_days)

def pretty_days(tot_days):
    if tot_days < 0:
        return "Days can't be negative"
    elif tot_days == 0:
        return "0 days"
    else:
        years = tot_days // 365
        days = tot_days % 365
        if years * days != 0:
            connector = " and "
        else:
            connector = " "
        if years == 0:
            num_years = ""
            year_s = ""
        elif years == 1:
            num_years = 1
            year_s = " year"
        else:
            num_years = years
            year_s = " years"
        
        if days == 0:
            num_days = ""
            day_s = ""
        elif days == 1:
            num_days = 1
            day_s = " day "
        else:
            num_days = days
            day_s = " days "
        return str(num_years) + year_s + connector + str(num_days) + day_s

def target_rate(current_rate, current_days, target_days):
    '''
    rate: int or float, e.g. 0.03, yearly rate
    days: int or float, e.g. 53, number of days
    Takes rate% over days days to cagr% over 1 year
    '''
    return (1 + current_rate) ** (target_days / current_days) - 1

def download_assets(tickers, start_date=None, end_date=None, interval="1d"):
    """
    Download historical price data for multiple tickers from Yahoo Finance,
    and return a single DataFrame with a 'Ticker' column.
    
    Parameters:
        tickers (list of str): List of ticker symbols, e.g. ["AAPL", "AMZN"]
        start_date (str or None): Start date in "YYYY-MM-DD" format or None
        end_date (str or None): End date in "YYYY-MM-DD" format or None
        interval (str): Data interval, e.g. "1d", "1h", etc.
    
    Returns:
        pd.DataFrame: Combined DataFrame with multi-ticker data
    """
    all_data = []

    for ticker in tickers:
        df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
        df = df.reset_index()  # Move Date from index to column
        df["Ticker"] = ticker
        all_data.append(df)

    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

def download_sofr(start_date, end_date):
    fred = Fred(api_key="YOUR_API_KEY")
    sofr = fred.get_series("SOFR", observation_start=start_date, observation_end=end_date)

    # Make it daily, forward-fill missing weekends/holidays
    sofr = sofr.asfreq("D").ffill().rename("SOFR")

    return sofr

class Backtest:
    def __init__(self, portfolio, strategies, data, start_date, end_date):
        """
        portfolio: Portfolio object
        strategies: dict mapping ticker -> strategy instance
        data: DataFrame with ["Date", "Ticker", "Close", ...]
        start_date: pandas datetime object for start of the simulation
        end_date: pandas datetime object for end of the simulation
        """
        self.start_date = start_date
        self.end_date = end_date
        self.initial_value = portfolio.initial_value
        self.initial_bank_cash = portfolio.initial_value
        self.portfolio = portfolio
        self.strategies = strategies
        self.data = data.sort_values(["Date", "Ticker"])
        self.tot_buys = 0
        self.tot_sells = 0
        self.trade_log = []

    def run(self):

        for date, group in self.data.groupby("Date"):
            for ticker, asset_data in group.groupby("Ticker"):
                if ticker not in self.strategies:
                    continue  # skip tickers without a strategy

                strategy = self.strategies[ticker]
                signal = strategy.generate_signals(asset_data)
                last_signal = signal.iloc[-1]
                price = asset_data["Close"].iloc[-1]

                if last_signal == 1:
                    self.portfolio.buy(ticker, price, quantity=10)
                elif last_signal == -1:
                    self.portfolio.sell(ticker, price, quantity=10)
        return self
    
    def results(self):
        # Make sure dates are datetime objects
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)

        days = (pd.to_datetime(self.end_date) - pd.to_datetime(self.start_date)).days
        avg_daily_buys = round(self.tot_buys / days , 2)
        avg_daily_sells = round(self.tot_sells / days, 2)
        return {
            "portfolio": self.portfolio,
            "initial value": self.initial_value,
            "final cash": self.portfolio.cash,
            "total buys": self.tot_buys,
            "total sells": self.tot_sells,
            "avg daily buys": avg_daily_buys,
            "avg daily sells": avg_daily_sells,
            "days": days
        }

def backtester_with_dates(portfolio, strategies, data, start_date="2017-01-01", end_date="2025-08-15"):
    '''
    portfolio: Portfolio object
    strategies: dictionary with strategies for each asset
    data: data
    start_date: str object e.g. "YYYY-MM-DD"
    end_date: str object e.g. "YYYY-MM-DD"
    '''

    # Identify errors and problems ("if True" statement used to collapse the code)
    if True:
        # We will append all errors to this list so they can be displayed to the user
        errors = ["List of errors:"]

        # We take care of data-type for portfolio, strategies, data
        if not isinstance(portfolio, Portfolio):
            errors.append("The 'portfolio' object must be an instance of the Portfolio class.")
        if not isinstance(strategies, dict):
            errors.append("The 'strategies' object must be a Python dictionary.")
        if not isinstance(data, pd.DataFrame):
            errors.append("The 'data' object must be a pandas dataframe.")
        
        # We take care of start_date
        if not isinstance(start_date, dateformat):
            errors.append("The 'start_date' object must be a dateformat.")
            status_start_date = "bad"
        else:
            status_start_date = "good"
        
        # We take care of end_date
        if not isinstance(end_date, dateformat):
            errors.append("The 'end_date' object must be a dateformat.")
            status_end_date = "bad"
        else:
            status_end_date = "good"

        # We take care of the relationship between start_date and end_date
        if status_start_date == "good" and status_end_date == "good" and end_date - start_date <= 2:
            errors.append("The 'max_date' must be at least 3 days after the 'min_date'.")
        

    # If any errors are detected, we return the errors instead of running the simulation
    if len(errors) >= 2:
        return errors
    
    # Make sure dates are datetime objects
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Run the simulation
    results = Backtest(portfolio, strategies, data, start_date, end_date).run().results()

    # Calculate the investment return along the full period
    if True:
        if results["initial value"] == results["final cash"]:
            roi = 0
            rounded_roi = 0
        elif results["initial value"] == 0 and results["final cash"] > 0:
            roi = "infinity"
            rounded_roi = "infinity"
        elif results["initial value"] == 0 and results["final cash"] < 0:
            roi = "-infinity"
            rounded_roi = "-infinity"
        else:
            roi = results["final cash"] / results["initial value"] - 1
            rounded_roi = 100 * round(roi, 4)

        # Calculate the investment CAGR (annualised)
        if roi == "infinity" or roi == "-infinity":
            cagr = roi
            rounded_cagr = roi
        elif roi < -1:
            cagr = "NA"
            rounded_cagr = "NA"
        else:
            cagr = target_rate(roi, results["days"], 365)
            rounded_cagr = 100 * round(cagr, 4)



    # Calculate the bank return along the full period
    if True:
        if results["initial value"] == results["final bank cash"]:
            bank_roi = 0
            rounded_bank_roi = 0
        elif results["initial value"] == 0 and results["final bank cash"] > 0:
            bank_roi = "infinity"
            rounded_bank_roi = "infinity"
        elif results["initial value"] == 0 and results["final bank cash"] < 0:
            bank_roi = "-infinity"
            rounded_bank_roi = "-infinity"
        else:
            bank_roi = results["final bank cash"] / results["initial value"] - 1
            rounded_bank_roi = 100 * round(bank_roi, 4)

        # Calculate the bank CAGR (annualised)
        if bank_roi == "infinity" or bank_roi == "-infinity":
            bank_cagr = bank_roi
            rounded_bank_cagr = bank_roi
        elif bank_roi < -1:
            bank_cagr = "NA"
            rounded_bank_cagr = "NA"
        else:
            bank_cagr = target_rate(bank_roi, results["days"], 365)
            rounded_bank_cagr = 100 * round(bank_cagr, 4)
    
    # Store length of trading period in pretty format for presentation
    period = pretty_days(results["days"])

    return {
        "start date": start_date,
        "end date": end_date,
        "initial value": results["initial value"],
        "final cash": results["final cash"],
        "roi": rounded_roi,
        "cagr": rounded_cagr,
        "bank roi": rounded_bank_roi,
        "bank cagr": rounded_bank_cagr,
        "period": period,
        "total buys": results["total buys"],
        "total sells": results["total sells"],
        "total trades": results["total buys"] + results["total sells"],
        "avg daily buys": results["avg daily buys"],
        "avg daily sells": results["avg daily sells"],
        "avg daily trades": results["avg daily buys"] + results["avg daily sells"]
    }


def backtester_random_dates(portfolio, strategies, data, min_date = "2017-01-01", max_date = "2025-08-15", min_period=3, max_period=None, sample_size = 100):

    # Identify errors and problems ("if True" statement used to collapse the code)
    if True:
        # We will append all errors to this list so they can be displayed to the user
        errors = ["List of errors:"]

        # We take care of data-type for portfolio, strategies, data
        if not isinstance(portfolio, Portfolio):
            errors.append("The 'portfolio' object must be an instance of the Portfolio class.")
        if not isinstance(strategies, dict):
            errors.append("The 'strategies' object must be a Python dictionary.")
        if not isinstance(data, pd.DataFrame):
            errors.append("The 'data' object must be a pandas dataframe.")
        
        # We take care of min_date
        if not isinstance(min_date, dateformat):
            errors.append("The 'min_date' object must be a dateformat.")
            status_min_date = "bad"
        else:
            status_min_date = "good"
        
        # We take care of max_date
        if not isinstance(max_date, dateformat):
            errors.append("The 'max_date' object must be a dateformat.")
            status_max_date = "bad"
        else:
            status_max_date = "good"

        # We take care of the relationship between min_date and max_date
        if status_min_date == "good" and status_max_date == "good":
            dates_relative_status = "good"
            if max_date - min_date <=2:
                errors.append("The 'max_date' must be at least 3 days after the 'min_date'.")
        

        # We take care of min_period and its relationship with min_date-to-max_date period.
        if not isinstance(min_period, int):
            errors.append("The 'min_period' object must be an integer.")
            min_period_status = "bad"
        elif min_period <= 2:
            errors.append("The 'min_period' object must be at least 3.")
            min_period_status = "bad"
        elif dates_relative_status == "good" and max_date - min_date < min_period:
            errors.append("The 'min_period' is longer than the time between 'min_date' and 'max_date'.")
        else:
            min_period_status = "good"

        # We take care of max_period
        if  max_period != None:
            if not isinstance(max_period, int):
                errors.append("The 'max_period' object must be an integer or remain empty (no max period).")
            elif max_period <= 2:
                errors.append("The 'max_period' object must be at least 3 or remain empty (no max period).")
            elif min_period_status == "good" and max_period < min_period:
                errors.append("The 'max_period' object must be at least 'min_period' or remain empty (no max period).")
        
        # We take care of sample_size
        if not isinstance(sample_size, int):
            errors.append("The 'sample_size' object must be an integer or remain empty (will resort to a default value of 100).")
        elif sample_size <= 0:
            errors.append("The 'sample_size' object must be at least 1 or remain empty (will resort to a default value of 100).")

    # If any errors are detected we return the errors instead of running the simulations
    if len(errors) >= 2:
        return errors


    # Start parameters for later performance evaluation
    roi_s = []
    aagr_s = []
    cagr_s = []
    periods = []
    over_zero_counter = 0
    over_bank_counter = 0

    # Run simulations
    for i in range(sample_size):
        # Generate random start and end dates for this simulation
        start_date = random_date(min_date, max_date, (pd.to_datetime(max_date) - pd.to_datetime(min_date)).days - min_period)
        end_date = random_date(start_date + pd.Timedelta(days=min_period), max_date, max_period)
        
        # Store length of trading period
        period = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days / 365
        periods.append((pd.to_datetime(end_date) - pd.to_datetime(start_date)).days)


        #Only keep 
        # Run a backtest
        results = Backtest(portfolio, strategies, data, start_date, end_date).run()

        # Calculate the investment return
        roi = results['final cash'] / portfolio.cash - 1

        # Calculate the investment CAGR
        cagr = (1 + roi) ** (1 / period) - 1 or target_rate(roi, ?results["days"], 365)
        rounded_cagr = 100 * round(cagr, 4)

        # Calculate the bank return
        bank_roi = results['final bank cash'] / portfolio.cash - 1

        # Store parameters
        roi_s.append(roi)
        if roi > 0:
            over_zero_counter += 1
        if roi > bank_roi:
            over_bank_counter += 1
    # Find average profit
    avg_roi = roi_s.mean()
    return {
        "sample size": sample_size,
        "average roi": avg_roi,
        "over bank counter": over_bank_counter,
        "over zero counter": over_zero_counter
    }



# CHOICE

If you want to test for PRECISE DATES in the past, read PRECISE DATES SIMULATION cell (just below).
If you want to run a wider simulation with RANDOM DATES, go straight to RANDOM DATES cell.

In [ ]:
# PRECISE DATES SIMULATION: Follow the instructions.

# STEP 1. Modify the start date and end date of your trading period to your liking (they must be at least 3 days apart).
start_date = "2017-01-01"
end_date = "2018-02-01"

# STEP 2. Choose what assets you want to trade on
#         Place each asset inside exactly one strategy in the square brackets.
#         You can also modify the Take Profit and Stop Loss values for the strategy.

### Strategy 1: Moving average crossover (20 days vs 50 days)
assets_strat_1 = {
    "tickers": ["AAPL", "AMZN"], # Insert asset symbols in the square brackets (use comma to separate, leave empty [] if you don't want to use the strategy)
    "take profit": 0.1, #e.g. 0.15 is 15%
    "stop loss": 0.05   #e.g. 0.08 is 8%
}

### Strategy 2: RSI reversal 14
assets_strat_2 = {
    "tickers": ["BTC"],  # Insert asset symbols in the square brackets (use comma to separate, leave empty [] if you don't want to use the strategy)
    "take profit": 0.05, #e.g. 0.2 is 20%
    "stop loss": 0.01    #e.g. 0.08 is 8%
}

### Strategy 3: RSI reversal 10
assets_strat_3 = {
    "tickers": [], # Insert asset symbols in the square brackets (use comma to separate, leave empty [] if you don't want to use the strategy)
    "take profit": 0.01, #e.g. 0.15 is 15%
    "stop loss": 0.01    #e.g. 0.08 is 8%
}

# STEP 3. Choose how much cash ($) and what assets are in your starting portfolio, and how much you bought/sold them for.
#         Negative quantities mean you sold the asset, positive quantities mean you bought the asset.
#         If you want to start with no assets, just choose your amount of cash and set initial_positions = {}

initial_cash = 1000
initial_positions = {
    "AAPL": {
        "quantity": 10,
        "price": 500.9
    },
    "AMZN": {
        "quantity": -1,
        "price": 398.98
    }
                      
}

# Ignore the next bit and go down to the text starting with "BRILLIANT"

all_assets = list(set(assets_strat_1["tickers"]) | set(assets_strat_2["tickers"]) | set(assets_strat_3["tickers"]))

portfolio = Portfolio(initial_cash, initial_positions)

# Assign strategies to tickers
strategies = {}
for asset in all_assets:
    if asset in assets_strat_1:
        strategies[asset] = MovingAverageCrossover(short_window=10, long_window=50)
    elif asset in assets_strat_2:
        strategies[asset] = RSIReversal(period=14)
    elif asset in assets_strat_3:
        strategies[asset] = RSIReversal(period=10)

# Step 4: Download and create all relevant data
data = download_assets(all_assets, start_date, end_date, "1d")




# BRILLIANT
You've inputted all your necessary data. Do the following:
1. Run the cell above (the one you were modifying).
2. Scroll until you find a cell titled "PRECISE DATES RESULTS" (just below) and simply run it.
3. Read results on the Terminal!

In [ ]:
####### PRECISE DATES RESULT ########
# Run this cell and read results from the terminal


test = backtester_with_dates(portfolio, strategies, data, start_date, end_date)

#"if True" statement used to collapse code
if True:
    # If errors were raised we display them
    if isinstance(test, list):
        for error in test:
            print(error)
    
    else:
        # If no errors were raised we display the results
        print(f"We started with a portfolio value of {portfolio.initial_cash}$ on {test['start date']} and finished with {portfolio.cash}$ on {test['end date']}.")
        if test['roi'] > test['bank roi'] >= 0: # Portfolio positive bigger than bank nonnegative, P>B>=0
            print(f'Our profit of +{test['roi']}% was larger than the projected risk-free interest of +{test['bank roi']}%.')
        elif test['roi'] >= 0 > test['bank roi']: # Portfolio nonnegative bigger than bank negative, P>=0>B
            print(f'Our profit of +{test['roi']}% was larger than the projected risk-free interest of -{test['bank roi']}%.')
        elif test['roi'] == test['bank roi'] >=0: # Both perform equally nonnegatively, B=P>=0
            print(f'Our profit of +{test['roi']}% was the same as the projected risk-free interest.')
        elif test['roi'] == test['bank roi'] < 0: # Both perform equally negatively, 0>B=P
            print(f'Our loss of {test['roi']}% was the same as the projected risk-free interest.')
        elif test['bank roi'] > test['roi'] >= 0: # Portfolio nonnegative smaller than bank positive, B>P>=0
            print(f'Our profit of +{test['roi']}% was smaller than the projected risk-free interest of +{test['bank roi']}%.')
        elif test['bank roi'] >= 0 > test['roi']: # Portfolio negative, bank nonnegative, B>=0>P
            print(f'We lost {test['roi']}% while the risk-free interest was {test['bank roi']}%.')
        elif test['bank roi'] < test['roi'] <= 0: # Bank negative, smaller than portfolio nonpositive, 0>=P>B
            print(f'We lost {test['roi']}% while the risk-free interest was -{test['bank roi']}%.')
        elif test['roi'] < test['bank roi'] <= 0: # Bank nonpositive, bigger than portfolio negative, 0>=B>P
            print(f'We lost {test['roi']}% while the risk-free interest was -{test['bank roi']}%.')
        else:
            print("We didn't take this case into account, please update the code to include this combination.")
        
        if isinstance(test["cagr"], int) or isinstance(test["cagr"], float): # Print the CAGR if it makes sense
            print(f"In yearly terms, the portfolio performed with a CAGR of {test["cagr"]}%.")
        print(f"During {test["period"]} we performed {test["total buys"]} buy trades and {test["total sells"]} for a total of {test["total trades"]}.") # Print number of trades per type
        print(f"That's an average {test["avg daily buys"]} daily buy trades and {test["avg daily sells"]} daily sell trades for a total of {test["avg daily trades"]} daily trades.") # Print number of trades per type per day
        

In [ ]:
# RANDOM DATES: Follow the instructions.

# STEP 1. Modify the number of simulations to your liking. More simulations give more precision but take more time.
sample_size = 100

# STEP 2. Modify the minimum start date and the maximum end date for all the simulations (they must be at least 3 days apart)..
min_date = "2017-01-01"
max_date = "2018-02-01"

# STEP 3. How long would you want your trading sessions to last? Give a minimum and maximum value (in days).
#         min_period has to be at least 3, max_period is 

min_period = 20
max_period = 368


# STEP 2. Choose what assets you want to trade on
#         Place each asset inside exactly one strategy in the square brackets.
#         You can also modify the Take Profit and Stop Loss values for the strategy.

### Strategy 1: Moving average crossover (20 days vs 50 days)
assets_strat_1 = {
    "tickers": ["AAPL", "AMZN"], # Insert asset symbols in the square brackets (use comma to separate, leave empty [] if you don't want to use the strategy)
    "take profit": 0.1, #e.g. 0.15 is 15%
    "stop loss": 0.05   #e.g. 0.08 is 8%
}

### Strategy 2: RSI reversal 14
assets_strat_2 = {
    "tickers": ["BTC"],  # Insert asset symbols in the square brackets (use comma to separate, leave empty [] if you don't want to use the strategy)
    "take profit": 0.05, #e.g. 0.2 is 20%
    "stop loss": 0.01    #e.g. 0.08 is 8%
}

### Strategy 3: RSI reversal 10
assets_strat_3 = {
    "tickers": [], # Insert asset symbols in the square brackets (use comma to separate, leave empty [] if you don't want to use the strategy)
    "take profit": 0.01, #e.g. 0.15 is 15%
    "stop loss": 0.01    #e.g. 0.08 is 8%
}

# STEP 3. Choose how much cash ($) you have at the start of every simulation.

initial_cash = 1000

# Ignore the next bit and go down to the text starting with "FANTASTIC"

all_assets = list(set(assets_strat_1["tickers"]) | set(assets_strat_2["tickers"]) | set(assets_strat_3["tickers"]))

portfolio = Portfolio(initial_cash, {})

# Assign strategies to tickers
strategies = {}
for asset in all_assets:
    if asset in assets_strat_1:
        strategies[asset] = MovingAverageCrossover(short_window=10, long_window=50)
    elif asset in assets_strat_2:
        strategies[asset] = RSIReversal(period=14)
    elif asset in assets_strat_3:
        strategies[asset] = RSIReversal(period=10)

# Step 4: Download and create all relevant data
data = 0

# FANTASTIC
You've inputted all your necessary data. Do the following:
1. Run the cell above (the one you were modifying).
2. Scroll until you find a cell titled "RANDOM DATES RESULTS" (just below) and simply run it.
3. Read results on the Terminal!

In [ ]:
####### RANDOM DATES RESULT ########
test = backtester_random_dates(portfolio, strategies, data, min_date, max_date, min_period, max_period, sample_size)

# If errors were raised we display them
if isinstance(test, list):
    for error in test:
        print(error)

# If no errors were raised we display the results
else:
    print(f"We ran {test['sample size']} simulations with this strategy, starting with {portfolio.initial_cash}$ in cash and the following stocks:")
    for stock in portfolio.initial_positions:
        print(f"{portfolio.initial_positions[stock]['quantity']} shares of {stock} each priced at {portfolio.initial_positions[stock]['price']}.")

    print(f"On average, the profit was {round(100 * test['average roi'], 2)}%.")
    print(f"{round(100 * test['over bank counter'] / test['sample size'], 2)}% of the simulations ended with more profit than an equivalent bank account.")
    print(f"{round(100 * test['over zero counter'] / test['sample size'], 2)}% of the simulations ended with positive profit.")

In [ ]:
# IF WE DID THE RANDOM BACKTESTER USING THE ORIGINAL FUNCTION

def backtester_random_dates(portfolio, strategies, data, min_date = "2017-01-01", max_date = "2025-08-15", min_period=3, max_period=None, sample_size = 100):

    # Identify errors and problems ("if True" statement used to collapse the code)
    if True:
        # We will append all errors to this list so they can be displayed to the user
        errors = ["List of errors:"]

        # We take care of data-type for portfolio, strategies, data
        if not isinstance(portfolio, Portfolio):
            errors.append("The 'portfolio' object must be an instance of the Portfolio class.")
        if not isinstance(strategies, dict):
            errors.append("The 'strategies' object must be a Python dictionary.")
        if not isinstance(data, pd.DataFrame):
            errors.append("The 'data' object must be a pandas dataframe.")
        
        # We take care of min_date
        if not isinstance(min_date, dateformat):
            errors.append("The 'min_date' object must be a dateformat.")
            status_min_date = "bad"
        else:
            status_min_date = "good"
        
        # We take care of max_date
        if not isinstance(max_date, dateformat):
            errors.append("The 'max_date' object must be a dateformat.")
            status_max_date = "bad"
        else:
            status_max_date = "good"

        # We take care of the relationship between min_date and max_date
        if status_min_date == "good" and status_max_date == "good":
            dates_relative_status = "good"
            if max_date - min_date <=2:
                errors.append("The 'max_date' must be at least 3 days after the 'min_date'.")
        

        # We take care of min_period and its relationship with min_date-to-max_date period.
        if not isinstance(min_period, int):
            errors.append("The 'min_period' object must be an integer.")
            min_period_status = "bad"
        elif min_period <= 2:
            errors.append("The 'min_period' object must be at least 3.")
            min_period_status = "bad"
        elif dates_relative_status == "good" and max_date - min_date < min_period:
            errors.append("The 'min_period' is longer than the time between 'min_date' and 'max_date'.")
        else:
            min_period_status = "good"

        # We take care of max_period
        if  max_period != None:
            if not isinstance(max_period, int):
                errors.append("The 'max_period' object must be an integer or remain empty (no max period).")
            elif max_period <= 2:
                errors.append("The 'max_period' object must be at least 3 or remain empty (no max period).")
            elif min_period_status == "good" and max_period < min_period:
                errors.append("The 'max_period' object must be at least 'min_period' or remain empty (no max period).")
        
        # We take care of sample_size
        if not isinstance(sample_size, int):
            errors.append("The 'sample_size' object must be an integer or remain empty (will resort to a default value of 100).")
        elif sample_size <= 0:
            errors.append("The 'sample_size' object must be at least 1 or remain empty (will resort to a default value of 100).")

    # If any errors are detected we return the errors instead of running the simulations
    if len(errors) >= 2:
        return errors


    # Start parameters for later performance evaluation
    roi_s = []
    over_zero_counter = 0
    over_bank_counter = 0

    # Run simulations
    for i in range(sample_size):
        # Generate random start and end dates for this simulation
        start_date = random_date(min_date, max_date, (pd.to_datetime(max_date) - pd.to_datetime(min_date)).days - min_period)
        end_date = random_date(start_date + pd.Timedelta(days=min_period), max_date, max_period)
        

        # Run simple backtester function with acquired random dates
        test = backtester_with_dates(portfolio, strategies, data, start_date=start_date, end_date=end_date)

        # Calculate the investment return
        roi = test['roi'] / portfolio.cash - 1

        # Calculate the bank return
        bank_roi = test['bank roi'] / portfolio.cash - 1

        # Store parameters
        roi_s.append(roi)
        if roi > 0:
            over_zero_counter += 1
        if roi > bank_roi:
            over_bank_counter += 1
    # Find average profit
    avg_roi = roi_s.mean()
    return {
        "sample size": sample_size,
        "average roi": avg_roi,
        "over bank counter": over_bank_counter,
        "over zero counter": over_zero_counter
    }
